# Calculating the potential revenue of each market

In [237]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [238]:
df = pd.read_csv('../data/cleaned_data/eu_da_prices_clean.csv', index_col=0)
df.date = pd.to_datetime(df.date)
df = df.loc[df.date >= pd.to_datetime('2022-01-01')].reset_index(drop=True)
df.head()

,Austria,Belgium,Croatia,Czech,De-Lu,Denmark 1,Denmark 2,Estonia,Finland,France,...,Slovenia,Spain,Sweden 1,Sweden 2,Sweden 3,Sweden 4,Switzerland,date,datetime,hour
0,61.84,82.02,61.84,50.05,78.44,50.05,46.60,50.05,46.60,89.06,...,61.84,145.86,46.60,46.60,46.60,46.60,119.97,2022-01-01,2022-01-01 00:00:00,1
1,51.44,67.07,51.44,41.33,69.75,41.33,41.33,41.33,41.33,78.48,...,51.44,114.90,41.33,41.33,41.33,41.33,101.54,2022-01-01,2022-01-01 01:00:00,2
2,55.91,75.11,55.91,43.22,68.04,43.22,42.18,42.18,42.18,85.16,...,55.91,113.87,42.18,42.18,42.18,42.18,52.13,2022-01-01,2022-01-01 02:00:00,3
3,46.94,50.91,46.94,45.46,51.27,45.46,44.37,44.37,44.37,50.00,...,46.94,97.80,44.37,44.37,44.37,44.37,20.78,2022-01-01,2022-01-01 03:00:00,4
4,37.67,37.67,37.67,37.67,42.46,37.67,37.67,37.67,37.67,37.67,...,37.67,97.80,37.67,37.67,37.67,37.67,15.66,2022-01-01,2022-01-01 04:00:00,5


### Single cycle 1 hour

In [239]:
cycle_1_hour_1 = df.drop(['hour', 'datetime'], axis=1).groupby('date').apply(lambda x: x.max() - x.min())
cycle_1_hour_1.head()

,Austria,Belgium,Croatia,Czech,De-Lu,Denmark 1,Denmark 2,Estonia,Finland,France,...,Romania,Serbia,Slovenia,Spain,Sweden 1,Sweden 2,Sweden 3,Sweden 4,Switzerland,date
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,112.30,112.30,112.30,112.30,107.41,112.30,112.30,112.30,112.30,112.30,...,99.27,86.97,112.30,120.76,13.25,13.25,112.30,112.30,133.23,0 days
2022-01-02,65.84,40.53,65.84,65.84,56.86,65.84,65.84,65.84,65.84,40.53,...,235.30,117.22,65.84,105.40,21.89,21.89,65.84,65.84,110.41,0 days
2022-01-03,141.42,163.95,285.29,141.42,137.56,139.47,130.33,128.67,119.75,180.30,...,173.00,192.43,285.29,88.90,28.85,28.85,119.75,119.75,223.51,0 days
2022-01-04,90.53,91.90,90.53,92.44,93.44,98.53,148.78,99.58,131.22,85.00,...,190.32,155.45,90.53,85.00,12.41,12.41,131.22,148.78,115.05,0 days
2022-01-05,104.18,165.17,104.18,94.62,113.59,98.56,98.56,84.63,84.63,189.62,...,126.09,86.43,104.18,174.96,27.12,27.12,98.56,98.56,151.03,0 days


In [240]:
cycle_1_hour_1_rev = cycle_1_hour_1.sum()
cycle_1_hour_1_rev = cycle_1_hour_1_rev.drop('date').astype(float).sort_values(ascending=False)
cycle_1_hour_1_rev

Estonia        89550.03
Lithuania      86886.37
Latvia         86331.48
Romania        85421.37
Greece         76838.35
Netherlands    74865.43
Hungary        74368.99
Sweden 4       73867.40
Finland        73432.29
Belgium        72030.46
Denmark 2      70884.84
Croatia        70608.34
Germany        68252.06
Sweden 3       67474.05
Slovenia       67368.71
Czech          66943.99
GB             65669.67
Denmark 1      65157.15
France         63843.47
Serbia         63552.16
Austria        59681.02
Italy          58954.95
De-Lu          54973.29
Poland         52234.22
Ireland        50453.39
Switzerland    46144.21
Norway 2       38806.75
Spain          34469.21
Portugal       33851.06
Norway 1       31746.80
Norway 5       30420.15
Sweden 2       23643.61
Sweden 1       21563.66
Norway 3       14615.68
Norway 4        8066.02
dtype: float64

### Dual cycle 1 hour

In [241]:
cycle_2_hour_1 = df.copy()
cycle_2_hour_1['am_pm'] = cycle_2_hour_1.hour.apply(lambda x: 'am' if x < 13 else 'pm')
cycle_2_hour_1_am = cycle_2_hour_1.loc[cycle_2_hour_1.am_pm == 'am']
cycle_2_hour_1_pm = cycle_2_hour_1.loc[cycle_2_hour_1.am_pm == 'pm']

In [242]:
cycle_2_hour_1 = cycle_2_hour_1.drop(['hour', 'datetime'], axis=1).groupby(['date', 'am_pm']).apply(lambda x: x.max() - x.min())
cycle_2_hour_1

Austria  Belgium  Croatia   Czech   De-Lu  Denmark 1  \
date       am_pm                                                         
2022-01-01 am       46.43    46.43    46.43   46.43   42.53      46.43   
           pm       64.81    64.81    64.81   64.81   56.64      66.86   
2022-01-02 am       40.53    40.53    40.53   40.53   40.13      40.53   
           pm       62.47    36.57    62.47   62.47   46.76      62.47   
2022-01-03 am      124.40   103.16   232.44  128.46  114.75     122.83   
...                   ...      ...      ...     ...     ...        ...   
2022-12-29 pm       48.06    51.91   100.97   45.18   37.01      49.42   
2022-12-30 am       54.63    29.09    56.37   45.97   34.99      28.51   
           pm       50.02    30.81    95.19   46.49   40.16      30.78   
2022-12-31 am       10.28     1.54    11.37    4.01   12.73       6.83   
           pm       15.36    27.80    83.90   29.80   28.42      63.26   

                  Denmark 2  Estonia  Finland  France  ...  Portugal  Romania  \
date       am_pm                                       ...                      
2022-01-01 am         46.43    46.43    45.96   51.39  ...     75.81    24.77   
           pm         66.86    64.81    64.81   64.81  ...     96.07    93.86   
2022-01-02 am         40.53    40.53    40.53   40.53  ...     22.69    60.39   
           pm         62.47    62.47    62.47   34.92  ...     99.30   212.07   
2022-01-03 am        113.69   115.71   103.11  180.30  ...     44.77   134.01   
...                     ...      ...      ...     ...  ...       ...      ...   
2022-12-29 pm         13.95   120.02    12.23   46.37  ...     22.50   337.85   
2022-12-30 am         28.51    94.51    28.51   38.40  ...      7.43    62.14   
           pm         30.77   105.91    30.77   30.16  ...     17.67   197.87   
2022-12-31 am          4.10     4.12     4.10    1.06  ...      1.06    18.01   
           pm         21.81    54.45    21.81   20.37  ...     10.99   196.70   

                  Serbia  Slovenia   Spain  Sweden 1  Sweden 2  Sweden 3  \
date       am_pm                                                           
2022-01-01 am      67.07     46.43   75.81     11.79     11.79     46.43   
           pm      62.53     64.81   96.07      4.18      4.18     66.86   
2022-01-02 am      88.29     40.53   29.51     20.45     20.45     40.53   
           pm      61.23     62.47  105.40     18.52     18.52     62.47   
2022-01-03 am     115.17    232.44   44.77     28.66     28.66    103.11   
...                  ...       ...     ...       ...       ...       ...   
2022-12-29 pm      61.74     70.15   22.50     12.23     12.23     12.23   
2022-12-30 am      82.95     53.39    7.43     28.51     28.51     28.51   
           pm     104.54     72.56   17.67     30.77     30.77     30.77   
2022-12-31 am      50.06      9.29    1.06      4.10      4.10      4.10   
           pm     115.86     46.86   19.90     21.81     21.81     21.81   

                  Sweden 4  Switzerland  
date       am_pm                         
2022-01-01 am        46.43       104.31  
           pm        66.86        78.17  
2022-01-02 am        40.53        72.67  
           pm        62.47        77.49  
2022-01-03 am       103.11       214.78  
...                    ...          ...  
2022-12-29 pm        12.23        37.51  
2022-12-30 am        28.51        32.40  
           pm        30.77        31.00  
2022-12-31 am         4.10        10.00  
           pm        21.81        15.68  

[730 rows x 35 columns]

In [243]:
cycle_2_hour_1_rev = cycle_2_hour_1.sum().sort_values(ascending=False)
cycle_2_hour_1_rev

Estonia        140843.65
Latvia         135008.54
Lithuania      134765.28
Romania        126538.62
Netherlands    118202.57
Greece         115454.81
Finland        114307.22
Sweden 4       112972.29
Belgium        112553.58
Hungary        111015.74
Denmark 2      105492.89
Croatia        105484.06
Sweden 3       105323.18
Germany        104827.20
Czech          104274.78
Slovenia       100646.10
Denmark 1       97049.84
GB              96427.40
France          93747.97
Serbia          92445.58
Austria         91186.56
Italy           86987.76
De-Lu           84288.07
Poland          76592.56
Ireland         70484.63
Switzerland     69728.75
Norway 2        58146.63
Spain           52218.84
Portugal        51180.50
Norway 1        47613.89
Norway 5        45728.59
Sweden 2        33558.24
Sweden 1        30666.29
Norway 3        20288.17
Norway 4        11429.60
dtype: float64

### Single cycle 2 hour

(Dis)charging times based on average price profile

In [244]:
hour_ave = df.groupby('hour').mean()
single_cycle_charging_times = dict()
single_cycle_discharging_times = dict()

for market in hour_ave.columns:
    min_hour = hour_ave[market].idxmin()
    second_min_price = min(hour_ave[market][min_hour-1], hour_ave[market][min_hour+1])
    second_min_hour = hour_ave[market].loc[hour_ave[market] == second_min_price].index.item()
    max_hour = hour_ave[market].idxmax()
    second_max_price = max(hour_ave[market][max_hour-1], hour_ave[market][max_hour+1])
    second_max_hour = hour_ave[market].loc[hour_ave[market] == second_max_price].index.item()

    single_cycle_charging_times[market] = [min_hour, second_min_hour]
    single_cycle_discharging_times[market] = [max_hour, second_max_hour]
    
single_cycle_charging_times

{'Austria': [4, 5],
 'Belgium': [14, 15],
 'Croatia': [4, 5],
 'Czech': [4, 5],
 'De-Lu': [4, 5],
 'Denmark 1': [15, 14],
 'Denmark 2': [4, 5],
 'Estonia': [4, 3],
 'Finland': [3, 4],
 'France': [5, 4],
 'GB': [4, 5],
 'Germany': [4, 5],
 'Greece': [4, 5],
 'Hungary': [4, 5],
 'Ireland': [5, 6],
 'Italy': [4, 5],
 'Latvia': [4, 3],
 'Lithuania': [4, 3],
 'Netherlands': [14, 15],
 'Norway 1': [4, 5],
 'Norway 2': [4, 5],
 'Norway 3': [4, 3],
 'Norway 4': [4, 3],
 'Norway 5': [4, 5],
 'Poland': [4, 3],
 'Portugal': [16, 17],
 'Romania': [4, 3],
 'Serbia': [4, 5],
 'Slovenia': [4, 5],
 'Spain': [16, 17],
 'Sweden 1': [4, 3],
 'Sweden 2': [4, 3],
 'Sweden 3': [4, 3],
 'Sweden 4': [4, 3],
 'Switzerland': [5, 4]}

In [245]:
def calculate_rev(df, hour_ave, charging_dict, discharging_dict):
    rev_dict = dict()
    for market in hour_ave.columns:
        chargin_cost = df.loc[df.hour.isin(charging_dict[market])][market].sum()
        dischargin_cost = df.loc[df.hour.isin(discharging_dict[market])][market].sum()
        rev_dict[market] = dischargin_cost - chargin_cost
    rev_df = pd.Series(rev_dict).sort_values(ascending=False)
    return rev_df
cycle_1_hour_2_rev = calculate_rev(df, hour_ave, single_cycle_charging_times, single_cycle_discharging_times)
cycle_1_hour_2_rev

Romania        122265.96
Hungary        109157.07
Estonia        106434.91
Latvia         105627.40
Croatia        105086.52
Serbia         103918.13
Lithuania      103527.63
GB             102684.70
Finland        100640.20
Slovenia       100491.01
Greece          98996.23
Sweden 4        98068.84
Sweden 3        94899.09
Netherlands     93562.84
France          90023.46
Czech           89933.57
Belgium         86047.09
Austria         85339.86
Italy           85073.47
Germany         83651.41
Denmark 2       81789.71
De-Lu           78160.33
Denmark 1       73885.56
Poland          71850.68
Ireland         71368.00
Switzerland     71004.23
Spain           48371.17
Portugal        47751.59
Norway 2        41564.46
Norway 1        35412.23
Norway 5        33843.51
Sweden 2        26745.43
Sweden 1        25484.86
Norway 3        14069.02
Norway 4         8627.91
dtype: float64

### Single cycle 1 hour based on preset times

(Dis)charging times based on average price profile

In [246]:
hour_ave = df.groupby('hour').mean()
single_cycle_1hr_charging_times = dict()
single_cycle_1hr_discharging_times = dict()

for market in hour_ave.columns:
    min_hour = hour_ave[market].idxmin()
    max_hour = hour_ave[market].idxmax()
    single_cycle_1hr_charging_times[market] = [min_hour]
    single_cycle_1hr_discharging_times[market] = [max_hour]
    
cycle_1_hour_1_rev_preset = calculate_rev(df, hour_ave, single_cycle_1hr_charging_times, single_cycle_1hr_discharging_times)
cycle_1_hour_1_rev_preset

Romania        65516.76
Hungary        58719.99
Croatia        56765.93
Latvia         56048.55
Lithuania      55482.23
Serbia         55119.71
Estonia        54889.32
Slovenia       54271.70
GB             53701.80
Greece         53467.39
Finland        52114.88
Sweden 4       49484.66
Netherlands    49459.67
Sweden 3       49011.17
Czech          48427.18
France         48341.74
Belgium        46217.34
Austria        45873.30
Germany        45353.43
Italy          45200.68
Denmark 2      43560.14
De-Lu          41458.34
Denmark 1      39566.18
Switzerland    37493.06
Ireland        36879.05
Poland         36227.82
Spain          25033.63
Portugal       24701.92
Norway 2       21415.79
Norway 1       18746.55
Norway 5       17954.32
Sweden 2       13614.78
Sweden 1       13168.00
Norway 3        7345.61
Norway 4        4384.81
dtype: float64

### Dual cycle 1 hour based on preset times

(Dis)charging times based on average price profile

In [247]:
hour_ave_am_charge = df.groupby('hour').mean().iloc[:6]
hour_ave_am_discharge = df.groupby('hour').mean().iloc[6:12]
hour_ave_pm_charge = df.groupby('hour').mean().iloc[12:17]
hour_ave_pm_discharge = df.groupby('hour').mean().iloc[16:24]

In [248]:
dual_cycle_1hr_charging_times = dict()
dual_cycle_1hr_discharging_times = dict()

for market in hour_ave.columns:
    min_hour_am = hour_ave_am_charge[market].idxmin()
    max_hour_am = hour_ave_am_discharge[market].idxmax()

    min_hour_pm = hour_ave_pm_charge[market].idxmin()
    max_hour_pm = hour_ave_pm_discharge[market].idxmax()

    dual_cycle_1hr_charging_times[market] = [min_hour_am, min_hour_pm]
    dual_cycle_1hr_discharging_times[market] = [max_hour_am, max_hour_pm]
    
cycle_2_hour_2_rev_preset = calculate_rev(df, hour_ave, dual_cycle_1hr_charging_times, dual_cycle_1hr_discharging_times)
cycle_2_hour_2_rev_preset

Romania        89586.43
Netherlands    81522.63
Hungary        81390.12
Lithuania      80999.75
Latvia         80880.21
Belgium        79103.44
Greece         78962.77
Germany        78823.34
Estonia        76340.95
Croatia        76278.18
Czech          75366.50
Denmark 2      74986.00
Slovenia       73978.49
Sweden 4       73937.36
GB             72586.78
France         71091.18
Serbia         70932.63
Denmark 1      70359.89
Sweden 3       68307.52
Italy          67791.77
Finland        67426.92
De-Lu          66273.52
Austria        64923.52
Switzerland    55283.33
Poland         49620.10
Ireland        48572.08
Spain          38757.33
Portugal       38216.80
Norway 2       37599.55
Norway 1       29457.09
Norway 5       28434.89
Sweden 2       15624.47
Sweden 1       14882.85
Norway 3        8549.24
Norway 4        4771.65
dtype: float64

### Dual cycle 2 hour

(Dis)charging times based on average price profile

In [249]:
hour_ave_am_charge = df.groupby('hour').mean().iloc[:6]
hour_ave_am_discharge = df.groupby('hour').mean().iloc[6:12]
hour_ave_pm_charge = df.groupby('hour').mean().iloc[12:17]
hour_ave_pm_discharge = df.groupby('hour').mean().iloc[16:24]

In [250]:
dual_cycle_charging_times = dict()
dual_cycle_discharging_times = dict()

for market in hour_ave.columns:
    min_hour_am = hour_ave_am_charge[market].idxmin()
    second_min_price_am = min(hour_ave_am_charge[market][min_hour_am-1], hour_ave_am_charge[market][min_hour_am+1])
    second_min_hour_am = hour_ave_am_charge[market].loc[hour_ave_am_charge[market] == second_min_price_am].index.item()
    max_hour_am = hour_ave_am_discharge[market].idxmax()
    second_max_price_am = max(hour_ave_am_discharge[market][max_hour_am-1], hour_ave_am_discharge[market][max_hour_am+1])
    second_max_hour_am = hour_ave_am_discharge[market].loc[hour_ave_am_discharge[market] == second_max_price_am].index.item()

    min_hour_pm = hour_ave_pm_charge[market].idxmin()
    second_min_price_pm = min(hour_ave_pm_charge[market][min_hour_pm-1], hour_ave_pm_charge[market][min_hour_pm+1])
    second_min_hour_pm = hour_ave_pm_charge[market].loc[hour_ave_pm_charge[market] == second_min_price_pm].index.item()
    max_hour_pm = hour_ave_pm_discharge[market].idxmax()
    try:
        second_max_price_pm = max(hour_ave_pm_discharge[market][max_hour_pm-1], hour_ave_pm_discharge[market][max_hour_pm+1])
        second_max_hour_pm = hour_ave_pm_discharge[market].loc[hour_ave_pm_discharge[market] == second_max_price_pm].index.item()
    except KeyError:  # Account for Norway 4's profile
        second_max_hour_pm = max_hour_pm + 1

    dual_cycle_charging_times[market] = [min_hour_am, second_min_hour_am, min_hour_pm, second_min_hour_pm]
    dual_cycle_discharging_times[market] = [max_hour_am, second_max_hour_am, max_hour_pm, second_max_hour_pm]
    
dual_cycle_charging_times

{'Austria': [4, 5, 15, 14],
 'Belgium': [4, 5, 14, 15],
 'Croatia': [4, 5, 14, 15],
 'Czech': [4, 5, 15, 14],
 'De-Lu': [4, 5, 15, 14],
 'Denmark 1': [4, 5, 15, 14],
 'Denmark 2': [4, 5, 15, 14],
 'Estonia': [4, 3, 14, 15],
 'Finland': [3, 4, 15, 14],
 'France': [5, 4, 16, 15],
 'GB': [4, 5, 15, 14],
 'Germany': [4, 5, 15, 14],
 'Greece': [4, 5, 14, 15],
 'Hungary': [4, 5, 14, 15],
 'Ireland': [5, 6, 16, 17],
 'Italy': [4, 5, 14, 15],
 'Latvia': [4, 3, 14, 15],
 'Lithuania': [4, 3, 14, 15],
 'Netherlands': [5, 4, 14, 15],
 'Norway 1': [4, 5, 15, 14],
 'Norway 2': [4, 5, 15, 14],
 'Norway 3': [4, 3, 15, 14],
 'Norway 4': [4, 3, 14, 13],
 'Norway 5': [4, 5, 15, 14],
 'Poland': [4, 3, 14, 15],
 'Portugal': [5, 4, 16, 17],
 'Romania': [4, 3, 14, 15],
 'Serbia': [4, 5, 14, 15],
 'Slovenia': [4, 5, 14, 15],
 'Spain': [5, 4, 16, 17],
 'Sweden 1': [4, 3, 16, 15],
 'Sweden 2': [4, 3, 15, 16],
 'Sweden 3': [4, 3, 15, 14],
 'Sweden 4': [4, 3, 15, 14],
 'Switzerland': [5, 4, 15, 14]}

In [251]:
cycle_2_hour_2_rev = calculate_rev(df, hour_ave, dual_cycle_charging_times, dual_cycle_discharging_times)
cycle_2_hour_2_rev

Romania        167740.92
Netherlands    154473.69
Hungary        152026.51
Latvia         151910.35
Lithuania      151203.16
Belgium        148824.17
Germany        147985.57
Estonia        146825.94
Greece         144067.19
Czech          141544.86
Sweden 4       141253.27
Croatia        140617.00
Denmark 2      140135.54
GB             136589.51
Slovenia       136507.05
Denmark 1      132450.91
Serbia         131854.59
France         129589.58
Sweden 3       129459.28
Finland        127576.53
De-Lu          124151.99
Italy          122916.79
Austria        120721.43
Switzerland    103073.66
Poland          95016.17
Ireland         93277.09
Spain           73965.41
Portugal        72975.49
Norway 2        70328.01
Norway 1        55436.93
Norway 5        53793.92
Sweden 2        30279.35
Sweden 1        28385.98
Norway 3        16095.33
Norway 4         8989.56
dtype: float64

### Plots

In [256]:
fig = make_subplots(
    rows=2, cols=1,
    shared_yaxes=True,
    subplot_titles=('Single cycle', 'Dual cycle')
)

fig.add_trace(go.Bar(
    x=cycle_1_hour_1_rev.index,
    y=cycle_1_hour_1_rev.values/1000,
    marker=dict(color=cycle_1_hour_1_rev.values, colorscale='inferno')
), row=1, col=1)

fig.add_trace(go.Bar(
    x=cycle_2_hour_1_rev.index,
    y=cycle_2_hour_1_rev.values/1000,
    marker=dict(color=cycle_2_hour_1_rev.values, colorscale='inferno'),
), row=2, col=1)

fig.update_yaxes(title='k€/MW')
fig.update_xaxes(tickangle=-45)
fig.update_traces(texttemplate='%{value: .3r}', textangle=270)
fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='Potential revenue for a 1 hour battery in 2022',
    yaxis_title='k€/MW',
    showlegend=False,
    template='plotly_white',
    width=1200,
    height=600
)

fig.write_image('../figures/1-hour-potential-rev.jpeg', scale=5, engine='orca')
fig

In [253]:
fig = make_subplots(
    rows=2, cols=1,
    shared_yaxes=True,
    subplot_titles=('Single cycle', 'Dual cycle')
)

fig.add_trace(go.Bar(
    x=cycle_1_hour_2_rev_preset.index,
    y=cycle_1_hour_2_rev_preset.values/1000,
    marker=dict(color=cycle_1_hour_2_rev_preset.values, colorscale='solar'),
), row=1, col=1)

fig.add_trace(go.Bar(
    x=cycle_2_hour_2_rev_preset.index,
    y=cycle_2_hour_2_rev_preset.values.reshape(-1)/1000,
    marker=dict(color=cycle_2_hour_2_rev_preset.values.reshape(-1), colorscale='solar'),
), row=2, col=1)

fig.update_xaxes(tickangle=-45)
fig.update_yaxes(title='k€/MW')
fig.update_traces(texttemplate='%{value: .3r}', textangle=270)
fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='Revenue for a 2 hour battery with preset flow hours in 2022',
    showlegend=False,
    template='plotly_white',
    width=1200,
    height=600
)

fig.write_image('../figures/1-hour-rev.jpeg', scale=5, engine='orca')
fig

In [254]:
fig = make_subplots(
    rows=2, cols=1,
    shared_yaxes=True,
    subplot_titles=('Single cycle', 'Dual cycle')
)

fig.add_trace(go.Bar(
    x=cycle_1_hour_2_rev.index,
    y=cycle_1_hour_2_rev.values/1000,
    marker=dict(color=cycle_1_hour_2_rev.values, colorscale='solar'),
), row=1, col=1)

fig.add_trace(go.Bar(
    x=cycle_2_hour_2_rev.index,
    y=cycle_2_hour_2_rev.values.reshape(-1)/1000,
    marker=dict(color=cycle_2_hour_2_rev.values.reshape(-1), colorscale='solar'),
), row=2, col=1)

fig.update_xaxes(tickangle=-45)
fig.update_yaxes(title='k€/MW')
fig.update_traces(texttemplate='%{value: .3r}', textangle=270)
fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='Revenue for a 2 hour battery with preset flow hours in 2022',
    showlegend=False,
    template='plotly_white',
    width=1200,
    height=600
)

fig.write_image('../figures/2-hour-rev.jpeg', scale=5, engine='orca')
fig